In [29]:
import cv2
from ultralytics import YOLO
import cvzone
import math

from sort import *

In [30]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [31]:
model = YOLO('./Yolo-Weights/yolov8l.pt')

In [32]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [33]:
mask = cv2.imread('./mask.png')

In [34]:
# creating a instance for tracking
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)

In [35]:
while True:
    success, img = cap.read()
    #overlay mask on the image/webcam feed
    imageRegion = cv2.bitwise_and(img,mask)
    # results = model(img,stream=True)
    results = model(imageRegion,stream=True)
    
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                # cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=5) #using cvzone rectange
                currentArrays = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArrays))
            
    resultsTracker = tracker.update(detections)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id = result
        # converting them into integers instead for tensor 
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h = x2-x1 , y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=2,colorR=(255,0,0))
        cvzone.putTextRect(img,f' {id}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imageRegion)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 766.9ms
Speed: 8.0ms preprocess, 766.9ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)



0.81
0.8
0.67
[        571         316         651         429          49]
[        378         348         495         457          48]
[        458         226         509         270          47]


0: 384x640 3 cars, 830.1ms
Speed: 8.0ms preprocess, 830.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



0.88
0.83
0.82
[     568.92      322.12      649.08      435.88          49]
[     373.98      352.02      493.02      462.98          48]
[     456.14      227.88      506.86      271.12          47]


0: 384x640 4 cars, 767.9ms
Speed: 0.0ms preprocess, 767.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

0.8
0.71
[     485.13      210.59      531.18      244.86          60]
[     489.48      261.99      555.46         320          58]


0: 384x640 2 cars, 743.9ms
Speed: 3.4ms preprocess, 743.9ms inference, 8.0ms postprocess per image at shape (1, 3, 384, 640)



0.79
0.74
[     478.31      215.98      526.36      252.97          60]
[      478.6      272.27      548.51      333.95          58]


0: 384x640 2 cars, 740.0ms
Speed: 8.0ms preprocess, 740.0ms inference, 8.4ms postprocess per image at shape (1, 3, 384, 640)


0.83
0.72
[     470.53      222.18      519.45      260.81          60]
[     466.71      284.41      540.21      349.51          58]
